`# TODO:`

* Documentation of this notebook
* Include example of how to add or remove features manually
* Include example of how to modify feature values manually


In [1]:
import pandas as pd
from gaishap import Featurizer
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
df_test_dataset = pd.read_json('./test-dataset.json', orient='records')
df_test_dataset.sample(5)

,user_input,retrieved_contexts,response,reference,faithfulness,context_precision,context_recall
89,How much has Microsoft invested in research an...,[PART IItem 2\n \nITEM 2. MANAGEMENT’S DISCUSS...,The specific amount Microsoft invested in rese...,"In the first quarter of 2023, Microsoft Corpor...",1.000000,0.0,0.000000
105,How much did NVIDIA allocate to research and d...,"[Cash, cash equivalents and marketable securit...",The provided context does not include specific...,"NVIDIA allocated $1,875 million to research an...",1.000000,0.0,0.000000
11,Has Microsoft partaken in any substantial stoc...,[PART IIItem 2\n \nITEM 2. UNREGISTERED SALES ...,Microsoft has engaged in substantial stock buy...,"Yes, Microsoft has partaken in substantial sto...",0.428571,1.0,0.142857
172,"In Intel's latest 10-Q, how does the revenue d...","[Table of Contents\nThe CODM, who is our CEO, ...",Intel's latest 10-Q indicates that revenue dis...,"In Intel's latest 10-Q, the revenue distributi...",0.818182,1.0,0.318182
126,Did Intel announce any significant changes to ...,[Table of Contents\nA Quarter in Review\nTotal...,There is no information provided about any sig...,"Yes, Intel announced a significant change to i...",0.000000,0.0,0.000000


In [4]:
featurizer = Featurizer.from_pandas(df_test_dataset)
featurizer.create_features_using_azure_openai(deployment_name="gpt-4o")
print(featurizer.features.model_dump_json(indent=4))

{
    "features": [
        {
            "feature": "there_is_any_company_indentified_in_the_question",
            "ftype": "boolean"
        },
        {
            "feature": "list_of_companies_identified_in_the_question",
            "ftype": "list_of_strings"
        },
        {
            "feature": "there_is_any_business_segment_indentified_in_the_question",
            "ftype": "boolean"
        },
        {
            "feature": "list_of_business_segments_identified_in_the_question",
            "ftype": "list_of_strings"
        },
        {
            "feature": "there_is_any_financial_metric_identified_in_the_question",
            "ftype": "boolean"
        },
        {
            "feature": "list_if_financial_metrics_identified_in_the_question",
            "ftype": "list_of_strings"
        },
        {
            "feature": "there_is_any_specific_quarter_identified_in_the_question",
            "ftype": "boolean"
        },
        {
            "feature": "list

In [5]:
featurizer.fill_out_features_using_azure_openai(deployment_name="gpt-4o")

  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
df_features = featurizer.to_pandas()
df_features.sample(5)

,there_is_any_company_indentified_in_the_question,list_of_companies_identified_in_the_question,there_is_any_business_segment_indentified_in_the_question,list_of_business_segments_identified_in_the_question,there_is_any_financial_metric_identified_in_the_question,list_if_financial_metrics_identified_in_the_question,there_is_any_specific_quarter_identified_in_the_question,list_of_quarters_identified_in_the_question,is_a_comparison_question,is_a_question_about_trends_or_changes_over_time,is_a_question_about_identification_of_factors_or_insights,is_a_question_related_to_the_notes_section
45,True,[Microsoft],True,[Cloud Services],True,[revenue],False,[],False,True,False,False
42,True,[Microsoft],False,[],True,[gross margin],True,[latest quarterly report],True,False,True,False
125,True,[Intel],False,[],True,[inventory levels],True,[Q3 2022],False,False,True,False
76,True,[Apple],False,[],False,[],True,[Q2 2023],False,False,True,True
180,True,[Intel],False,[],True,[capital expenditure],True,[Q3 2022],True,False,False,False


In [7]:
df_features.to_json('./test-features.json', orient='records', indent=4)

In [8]:
df_test_dataset.join(df_features).sample(5)

,user_input,retrieved_contexts,response,reference,faithfulness,context_precision,context_recall,there_is_any_company_indentified_in_the_question,list_of_companies_identified_in_the_question,there_is_any_business_segment_indentified_in_the_question,list_of_business_segments_identified_in_the_question,there_is_any_financial_metric_identified_in_the_question,list_if_financial_metrics_identified_in_the_question,there_is_any_specific_quarter_identified_in_the_question,list_of_quarters_identified_in_the_question,is_a_comparison_question,is_a_question_about_trends_or_changes_over_time,is_a_question_about_identification_of_factors_or_insights,is_a_question_related_to_the_notes_section
152,"In Microsoft's Q1 2023 10-Q, what relationship...",[PART IItem 2\n \n \nOperating income increase...,"In Microsoft's Q1 2023 10-Q, the research and ...",In the provided context of Microsoft's Q1 2023...,0.833333,0.0,1.000000,True,[Microsoft],False,[],True,[R&D spending],True,[Q1 2023],False,False,True,False
169,"From NVIDIA's Q3 2023 10-Q, how does the opera...",[NVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO...,"In NVIDIA's Q3 2023 10-Q, the operational expe...",The operational expenses section in NVIDIA's Q...,0.700000,1.0,0.500000,True,[NVIDIA],False,[],True,[operational expenses],True,[Q3 2023],True,False,True,False
144,"For Apple's Q2 2023 report, analyze the relati...",[Item 2. Management’s Discussion and Analys...,"In Apple's Q2 2023 report, the company experie...",Apple's Q2 2023 report indicates that internat...,1.000000,1.0,0.222222,True,[Apple],True,[international markets],True,[revenue],True,[Q2 2023],False,False,True,True
191,Analyze how Amazon's effective tax rate report...,[Table of Contents\nNote 7 — INCOME TAXES\nOur...,Amazon's effective tax rate for the most recen...,The effective tax rate for Amazon as reported ...,1.000000,1.0,0.571429,True,[Amazon],False,[],True,[effective tax rate],True,[most recent 10-Q],True,False,False,True
92,What details were provided about changes in Mi...,[PART IItem 1\n \n \nNOTE 10 — DEBT\nThe compo...,The Q1 2023 10-Q provided details about Micros...,"In the Q1 2023 10-Q, the details provided abou...",1.000000,0.0,0.000000,True,[Microsoft],False,[],True,"[debt, interest expenses]",True,[Q1 2023],False,False,True,False
